In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm import tqdm
import os 

import matplotlib.pylab as pl

In [2]:
import axisrules as axis

In [3]:
path_dataset = "../data/" 

list_datasets = []

for folder in os.listdir(path_dataset):
    for dataset in os.listdir(path_dataset+folder):
        candidates = path_dataset+folder+"/candidates.csv"
        # Test isdir 
        if os.path.isdir(path_dataset+folder+"/"+dataset):
            path_weights=  path_dataset+folder+"/"+dataset+"/weights.csv" 
            # Test if there is weight file:
            isweight = False
            if os.path.isfile(path_weights):
                isweight = True
            for file in os.listdir(path_dataset+folder+"/"+dataset+"/votes"):
                if file.endswith(".csv"):
                    list_datasets.append({"votes":path_dataset+folder+"/"+dataset+"/votes/"+file, 
                                            "weights":path_weights,
                                            "isweight":isweight,
                                            "candidates": candidates,
                                            "year": folder,
                                            "dataset": dataset,
                                            "file":file})



            

In [6]:
axis_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, 
              axis.MinimalSwaps, axis.ForbiddenTriples]
axis_rules_seq = [axis.SequentialVoterDeletion, axis.SequentialMinimalFlips, 
              axis.SequentialBallotCompletion, axis.SequentialMinimalSwaps, axis.SequentialForbiddenTriples]

n_candidates = 12

tab_res = []

df_previous = pd.read_csv("data/french_axis_no_w_%i.csv"%(n_candidates-1))
for data in list_datasets:


    # filter the one with dataset and year and file:
    df_previous_data =  df_previous[(df_previous["dataset"]==data["dataset"]) & (df_previous["year"]==data["year"]) & (df_previous["file"]==data["file"])]



    # print("")
    print(data["dataset"], data["year"])
    dataset = data["dataset"]
    year = data["year"]
    df = pd.read_csv(data["votes"], index_col=0)
    candidates = df.columns
    votes = df.values
    n_voters = votes.shape[0]
    # print("voters:",votes.shape[0],"\n")
    weights = None 
    df_cand = pd.read_csv(data["candidates"], index_col=0)

    if False:
        if data["isweight"]:
            df_weights = pd.read_csv(data["weights"], index_col=0)
            weights = []
            if data["file"] == "approval.csv":
                w_type = "approval"
            else:
                w_type = "approval_with_runoff"
            for i in range(len(df.index)):
                ind = df.index[i]
                if ind in df_weights.index:
                    weights.append(df_weights.loc[ind][w_type])
                else:
                    weights.append(0)
            weights = np.array(weights)
    

    file = data["file"]
    dict_cand = df_cand[["name", "score"]].to_dict()
    cands_id = [e[1] for e in sorted([(dict_cand["score"][x],x) for x in dict_cand["score"]])[::-1][:n_candidates]]
    cands_name = [dict_cand["name"][x] for x in cands_id]
    selected_id = [i for i in range(len(candidates)) if candidates[i] in cands_name]
    votes = votes[:, selected_id]
    # count number of votes with > 2 approvals 
    active_voters = np.sum(np.sum(votes, axis=1)>=2)
    candidates = [candidates[i] for i in selected_id]
    n_candidates_bis = len(candidates)
    if n_candidates != n_candidates_bis:
        print(n_candidates_bis, cands_name)
        if dataset != "IEP Paris":
            continue

    
    for j,rule in enumerate(axis_rules):
        name = rule.name
        rule_instance = rule(votes, weights)

        df_previous_rule = df_previous_data[df_previous_data["rule"]==name]

        # get first element
        previous_axis = df_previous_rule.iloc[0]["axis"]
        axis_num = axis.reverse_axis(previous_axis, cands_name[:-1])
        min_score = None
        min_axis = None
        for i in range(len(axis_num)+1):
            tested_axis = axis_num[:i]+[n_candidates-1]+axis_num[i:]
            score_axis = rule_instance.get_score(tested_axis)
            if min_score is None or score_axis < min_score:
                min_score = score_axis
                min_axis = tested_axis


        axis.print_axis(min_axis, cands_name)

        # seq_result = axis_rules_seq[j](votes, weights)()
        
        res = rule_instance.bruteforce(proxy_axis=min_axis)

        # axis.print_axis(res[0][0], candidates)
        axis_score = res[0][1]
        # print("Score of this axis :", int(axis_score))
        for el in res:
            tab_res.append([dataset, year, file, name, axis.get_axis(el[0], candidates), axis_score,n_voters,active_voters, n_candidates_bis])


df_res = pd.DataFrame(tab_res, columns=["dataset", "year", "file", "rule", "axis", "score", "n_voters","active_voters","n_candidates"])
df_res.to_csv("data/french_axis_no_w_%i.csv"%n_candidates,index=False)


Gy-Les-Nonains France 2002
Bruno Mégret < Jean-Pierre Chevènement < François Bayrou < Alain Madelin < Jacques Chirac < Jean Saint-Josse < Jean-Marie Le Pen < Arlette Laguiller < Robert Hue < Lionel Jospin < Noël Mamère < Olivier Besancenot
Jean Saint-Josse < Jean-Marie Le Pen < Jacques Chirac < Alain Madelin < François Bayrou < Bruno Mégret < Jean-Pierre Chevènement < Arlette Laguiller < Olivier Besancenot < Noël Mamère < Lionel Jospin < Robert Hue
Alain Madelin < François Bayrou < Jacques Chirac < Jean-Marie Le Pen < Jean-Pierre Chevènement < Jean Saint-Josse < Arlette Laguiller < Lionel Jospin < Noël Mamère < Bruno Mégret < Olivier Besancenot < Robert Hue
Jean-Pierre Chevènement < Alain Madelin < François Bayrou < Jacques Chirac < Jean-Marie Le Pen < Jean Saint-Josse < Arlette Laguiller < Lionel Jospin < Noël Mamère < Bruno Mégret < Olivier Besancenot < Robert Hue
Jean Saint-Josse < Alain Madelin < Jacques Chirac < Jean-Marie Le Pen < François Bayrou < Jean-Pierre Chevènement < Arlet

IndexError: index 11 is out of bounds for axis 0 with size 11

Voter Deletion
Arlette Laguiller < Olivier Besancenot < Noël Mamère < Lionel Jospin < Jean-Pierre Chevènement < François Bayrou < Jacques Chirac < Jean-Marie Le Pen
Score of this axis : 705
Minimal Flips
Arlette Laguiller < Olivier Besancenot < Noël Mamère < Lionel Jospin < Jean-Pierre Chevènement < François Bayrou < Jacques Chirac < Jean-Marie Le Pen
Score of this axis : 778
Ballot Completion
Arlette Laguiller < Olivier Besancenot < Noël Mamère < Lionel Jospin < Jean-Pierre Chevènement < François Bayrou < Jacques Chirac < Jean-Marie Le Pen
Score of this axis : 1214
Minimal Swaps
Arlette Laguiller < Olivier Besancenot < Noël Mamère < Lionel Jospin < Jean-Pierre Chevènement < François Bayrou < Jacques Chirac < Jean-Marie Le Pen
Score of this axis : 1272
Forbidden Triples
Arlette Laguiller < Olivier Besancenot < Noël Mamère < Lionel Jospin < Jean-Pierre Chevènement < François Bayrou < Jacques Chirac < Jean-Marie Le Pen
Score of this axis : 2205


In [34]:
axis_rules = [axis.VoterDeletion, axis.BallotCompletion]

for rule in axis_rules:
    print(rule.name)
    rule_instance = rule(votes, weights)
    res = rule_instance.lp()
    axis.print_axis(res, candidates)
    axis_score = rule_instance.get_score(res)   
    print("Score of this axis :", int(axis_score))

Voter Deletion


In [11]:
for data in list_datasets:
    

12